In [2]:
from pymongo import MongoClient
import pandas as pd
from rapidfuzz import fuzz, process
from itertools import permutations
import openai
from tqdm import tqdm
import time
import math
from datetime import datetime

In [4]:
docList = pd.read_excel(r'C:\Jupyter Notebook\Relevant Docs\final transcripts.xlsx')
docList = docList['Doc'].tolist()
topHundred = [s.lower().replace('.pdf','') for s in docList[50:100]]
print(len(topHundred))

50


In [5]:
client = MongoClient('mongodb://localhost:27017/')
db_pdf = client['transcripts']
collection_pdf = db_pdf['P1_speaker_speech']
data = list(collection_pdf.find({"file_name" : {"$in" : topHundred}}))
data = pd.DataFrame(data)
print(len(data))

19246


In [6]:
stakeholders = pd.read_excel(r'C:\Users\0132499s\OneDrive - National University of Ireland, Galway\Documents\Documents\stakeholder_category.xlsx')
stakeholders = stakeholders.fillna("")
stakeholders.replace({"\xa0": " "}, regex=True, inplace=True)
print(stakeholders)

                    Category  \
0                      Input   
1                      Input   
2                      Input   
3                      Input   
4                      Input   
..                       ...   
102        Trade association   
103  Research and Regulatory   
104  Research and Regulatory   
105  Research and Regulatory   
106  Research and Regulatory   

                                          Organisation  \
0           Milk Quality Ireland Co-operative Society    
1                                          Dairymaster   
2                                Animal Health Ireland   
3                                                Volac   
4                                       Grassland AGRO   
..                                                 ...   
102                       Dairy Industry Ireland (DII)   
103                            Enterprise Ireland (EI)   
104                   Science Foundation Ireland (SFI)   
105  Department of Agriculture, Fis

In [7]:
input_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Input']
production_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Production']
production_association_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Production - Associations']
production_smallcoop_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Production - Small co-operatives']
production_rawmilk_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Production - Raw Milk Producers']
prisec_smallbusiness_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Primary and Secondary Processing and Packaging - Small Businesses']
prisec_tradeassociation_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Primary and Secondary Processing and Packaging - Trade Association']
prisec_smallcoop_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Primary and Secondary Processing and Packaging - Small co-operatives']
prisec_largeglobal_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Primary and Secondary Processing and Packaging - Larger Globally operating Co-operatives/Businesses']
branding_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Branding, Marketing and Distribution']
sales_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Sales and Retail']
research_reg_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Research and Regulatory']
financial_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Financial Services']
ngo_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'NGO']
trade_category = [row['Organisation'] for _, row in stakeholders.iterrows() if row['Category'] == 'Trade association']

print(input_category)
print(production_category)
print(production_association_category)
print(production_smallcoop_category)
print(production_rawmilk_category)
print(prisec_smallbusiness_category)
print(prisec_tradeassociation_category)
print(prisec_smallcoop_category)
print(prisec_largeglobal_category)
print(branding_category)
print(sales_category)
print(research_reg_category)
print(financial_category)
print(ngo_category)
print(trade_category)

['Milk Quality Ireland Co-operative Society ', 'Dairymaster', 'Animal Health Ireland', 'Volac', 'Grassland AGRO']
['Small dairy farmer', 'Commercial dairy farmers', 'Mixed farmers', 'Small co-ops']
['Irish Farmers Association', 'Irish Creamery Milk Suppliers Association']
['Arrabawn Co-operative Society Ltd', 'Aurivo Co-operative Society Ltd', 'Bandon Co-operative Ltd', 'Barryroe Co-operative Ltd', 'Boherbue Co-operative Ltd', 'Callan Co-operative Ltd', 'Centenary Thurles Co-operative Ltd', 'Dairygold Co-operative Ltd', 'Drinagh Co-operative Ltd', 'Drombane Co-operative Ltd', 'Tirlán Co-operative Society Ltd', 'Kerry Co-operative Ltd', 'Lakeland Dairies Co-operative Ltd', 'Lee Strand Co-operative Ltd', 'Lisavaird Co-opertive Ltd', 'Maudabawn Co-operative Society Ltd', 'Mullinahone Co-operative Ltd', 'North Cork Co-operative Creameries Ltd', 'Ornua Co-operative Ltd', 'Tipperary Co-operative Ltd', 'Wexford Milk Producers Co-operative Society Ltd']
['St Tola Goats Milk (Inagh Farmhouse), 

In [8]:
def get_stakeholder_from_text(
    statement, 
    input_category,
    production_category,
    production_association_category,
    production_smallcoop_category,
    production_rawmilk_category,
    prisec_smallbusiness_category,
    prisec_tradeassociation_category,
    prisec_smallcoop_category,
    prisec_largeglobal_category,
    branding_category,
    sales_category,
    research_reg_category,
    financial_category,
    ngo_category,
    trade_category
    ):
    try:
        input_category_str = ", ".join(input_category)
        production_category_str = ", ".join(production_category)
        production_association_category_str = ", ".join(production_association_category)
        production_smallcoop_category_str = ", ".join(production_smallcoop_category)
        production_rawmilk_category_str = ", ".join(production_rawmilk_category)
        prisec_smallbusiness_category_str = ", ".join(prisec_smallbusiness_category)
        prisec_tradeassociation_category_str = ", ".join(prisec_tradeassociation_category)
        prisec_smallcoop_category_str = ", ".join(prisec_smallcoop_category)
        prisec_largeglobal_category_str = ", ".join(prisec_largeglobal_category)
        branding_category_str = ", ".join(branding_category)
        sales_category_str = ", ".join(sales_category)
        research_reg_category_str = ", ".join(research_reg_category)
        financial_category_str = ", ".join(financial_category)
        ngo_category_str = ", ".join(ngo_category)
        trade_category_str = ", ".join(trade_category)

        system_prompt = (
            "You are an assistant designed to analyze text and identify stakeholders "
            "based on predefined lists of stakeholders categorized into groups.\n\n"
            "Stakeholder Groups:\n"
                f"1. Input Category: {input_category_str}\n\n"
                f"2. Production Category: {production_category_str}\n\n"
                f"3. Production - Associations Category: {production_association_category_str}\n\n"
                f"4. Production - Small Co-operatives Category: {production_smallcoop_category_str}\n\n"
                f"5. Production - Raw Milk Producers Category: {production_rawmilk_category_str}\n\n"
                f"6. Primary and Secondary Processing and Packaging - Small Businesses Category: {prisec_smallbusiness_category_str}\n\n"
                f"7. Primary and Secondary Processing and Packaging - Trade Association Category: {prisec_tradeassociation_category_str}\n\n"
                f"8. Primary and Secondary Processing and Packaging - Small Co-operatives Category: {prisec_smallcoop_category_str}\n\n"
                f"9. Primary and Secondary Processing and Packaging - Larger Globally Operating Co-operatives/Businesses Category: {prisec_largeglobal_category_str}\n\n"
                f"10. Branding, Marketing and Distribution Category: {branding_category_str}\n\n"
                f"11. Sales and Retail Category: {sales_category_str}\n\n"
                f"12. Research and Regulatory Category: {research_reg_category_str}\n\n"
                f"13. Financial Services Category: {financial_category_str}\n\n"
                f"14. NGO Category: {ngo_category_str}\n\n"
                f"15. Trade association Category: {trade_category_str}\n\n"

            "When analyzing a statement:\n"
            "- Identify stakeholders explicitly mentioned in the provided lists and other possible stakeholder that may fall under any of the categories.\n"
            "- Do not extract names of individuals, titles, government or designations (e.g., 'Government', 'Mr. Cotter', 'Minister', 'Deputy Pringle').\n"
            "- If no valid stakeholders are mentioned in the statement, respond with: 'No stakeholders identified in this statement.'\n\n"
            "Provide your output in the following structured format:\n"
            "1. **Stakeholder Name**: (name of the identified stakeholder)\n"
            "2. **Category**: (name of the group the stakeholder belongs to, please respond with only the most suitable category name.)\n"
            "3. **Mention in Statement**: (exact sentence or phrase from the statement mentioning the stakeholder)\n\n"
            "If no stakeholders are identified, respond with: 'No stakeholders identified in this statement.'"
        )

        # User prompt
        user_prompt = (
            f"Analyze the following statement and identify stakeholders organisations based on the provided lists of categories:\n\n"
            f"Statement:\n\"{statement}\"\n\n"
            "List all stakeholders mentioned in the statement along with their categories. Do not extract names of individuals, titles, government or designations."
        )

        # API request
        response = openai.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=400,  
            temperature=0.2
        )
        content =  response.choices[0].message.content.strip()
        return content  # Directly return the response from the model

    except Exception as e:
        print(f"Error extracting names: {e}")
        return "Error processing the statement."

In [9]:
print("Start Time: ", datetime.now())
data["Extracted Stakeholders"] = data["Exact Text"].apply(
    lambda text: get_stakeholder_from_text(
        text, 
        input_category,
        production_category,
        production_association_category,
        production_smallcoop_category,
        production_rawmilk_category,
        prisec_smallbusiness_category,
        prisec_tradeassociation_category,
        prisec_smallcoop_category,
        prisec_largeglobal_category,
        branding_category,
        sales_category,
        research_reg_category,
        financial_category,
        ngo_category,
        trade_category
    )
)
print("End Time: ", datetime.now())

Start Time:  2025-04-01 05:40:49.469614
End Time:  2025-04-01 10:45:42.981754


In [10]:
data.head()

,_id,file_name,ID,Year,Sliding_year,Speaker,Exact Text,Category,Text,term_document_matrix,term_document_matrix_dept,Extracted Stakeholders,extracted_pol,Topic,Exact Text For Combined
0,671178d6b775f29fa839aeb9,"joint committee on agriculture, food and the m...",443,2021,2020-2022,Chairman,Apologies have been received from Deputy Fitzm...,"Agriculture, Food and Marine",Apologies puty Fitzmurice Senr remd members re...,"{'19': 1, 'action': 1, 'all': 2, 'and': 4, 'ap...","{'a': 1, 'action': 1, 'all': 2, 'and': 4, 'apo...",No stakeholders identified in this statement.,Nitrates Action Programme,NaN,NaN
1,671178d6b775f29fa839aeba,"joint committee on agriculture, food and the m...",443,2021,2020-2022,Chairman,The agenda for this morning’s meeting is the n...,"Agriculture, Food and Marine",gend morng meetg nitrtes ction progrmme consul...,"{'20': 1, 'about': 1, 'absolute': 2, 'action':...","{'20': 1, 'a': 6, 'about': 1, 'absolute': 2, '...",1. **Stakeholder Name**: Irish Farmers Associa...,new nitrates action programme,NaN,NaN
2,671178d6b775f29fa839aebb,"joint committee on agriculture, food and the m...",443,2021,2020-2022,Mr. Tim Cullinan,I compliment the Chairman on organising this e...,"Agriculture, Food and Marine",compliment Chirmn event morng be cuse time sub...,"{'10': 1, '14': 1, '16': 1, '2005': 1, '2008':...","{'1': 1, '10': 1, '14': 1, '16': 1, '2': 4, '2...",1. **Stakeholder Name**: Farmers \n **Categ...,,NaN,NaN
3,671178d6b775f29fa839aebc,"joint committee on agriculture, food and the m...",443,2021,2020-2022,Mr. Tim Cullinan,We are moving in the wrong direction there. W...,"Agriculture, Food and Marine",direction srge tnks sue compulsory low emsion ...,"{'100': 1, '15': 1, '30': 1, 'able': 1, 'above...","{'100': 1, '15': 1, '30': 1, 'a': 7, 'able': 1...",1. **Stakeholder Name**: Department\n **Cate...,"targeted agricultural modernisation scheme, TA...",NaN,NaN
4,671178d6b775f29fa839aebd,"joint committee on agriculture, food and the m...",443,2021,2020-2022,Mr. Pat McCormack,I thank committee members for the opportunity ...,"Agriculture, Food and Marine",committee members opportunity views behlf CMSA...,"{'000': 4, '10': 1, '100': 2, '12': 2, '120': ...","{'000': 4, '10': 1, '100': 2, '12': 1, '12-mon...",1. **Stakeholder Name**: ICMSA\n **Category*...,"TAMS, agricultural sustainability support and ...",NaN,NaN


In [11]:
filtered_data = data[data["Extracted Stakeholders"].str.strip() != ""]
print(filtered_data)

                            _id  \
0      671178d6b775f29fa839aeb9   
1      671178d6b775f29fa839aeba   
2      671178d6b775f29fa839aebb   
3      671178d6b775f29fa839aebc   
4      671178d6b775f29fa839aebd   
...                         ...   
19241  672bde88aa798a5c53f9b4c8   
19242  672bde89aa798a5c53f9b4c9   
19243  672bde89aa798a5c53f9b4ca   
19244  672bde89aa798a5c53f9b4cb   
19245  672bde89aa798a5c53f9b4cc   

                                               file_name   ID  Year  \
0      joint committee on agriculture, food and the m...  443  2021   
1      joint committee on agriculture, food and the m...  443  2021   
2      joint committee on agriculture, food and the m...  443  2021   
3      joint committee on agriculture, food and the m...  443  2021   
4      joint committee on agriculture, food and the m...  443  2021   
...                                                  ...  ...   ...   
19241       dáil éireann debate - wednesday, 19 may 2021  415  2021   
19242      

In [12]:
client = MongoClient('mongodb://localhost:27017/')
db_pdf = client['transcripts']
collection_pdf = db_pdf['P1_speaker_speech']

for _, row in data.iterrows():
    mongo_id = row["_id"]
    extracted_pol = row["extracted_pol"]
    
    collection_pdf.update_one(
        {"_id": mongo_id},
        {"$set": {"Extracted Stakeholders": extracted_pol}}
    )

In [13]:
def split_extracted_stakeholders(data, extracted_column = 'Extracted Stakeholders'):
    data['Stakeholder_Category'] = [[] for _ in range(len(data))]
    for idx, extracted_text in data[extracted_column].items():
        if isinstance(extracted_text, str):
            lines = extracted_text.split('\n')
            stakeholder_data = []
            stakeholder_name = None
            category = None
            mention = None
            for line in lines:
                if "**Stakeholder Name**:" in line:
                    stakeholder_name = line.split("**Stakeholder Name**:")[1].strip()
                elif "**Category**:" in line:
                    category = line.split("**Category**:")[1].strip()
                elif "**Mention in Statement**:" in line:
                    mention = line.split("**Mention in Statement**:")[1].strip()
                if stakeholder_name and category and mention:
                    stakeholder_data.append({
                        "Stakeholder" : stakeholder_name,
                        "Category" : category,
                        "Mention" : mention
                    })
                    stakeholder_name = category = mention = None
            data.at[idx, 'Stakeholder_Category'] = stakeholder_data
    return data

In [14]:
split_extracted_stakeholders(data)

,_id,file_name,ID,Year,Sliding_year,Speaker,Exact Text,Category,Text,term_document_matrix,term_document_matrix_dept,Extracted Stakeholders,extracted_pol,Topic,Exact Text For Combined,Stakeholder_Category
0,671178d6b775f29fa839aeb9,"joint committee on agriculture, food and the m...",443,2021,2020-2022,Chairman,Apologies have been received from Deputy Fitzm...,"Agriculture, Food and Marine",Apologies puty Fitzmurice Senr remd members re...,"{'19': 1, 'action': 1, 'all': 2, 'and': 4, 'ap...","{'a': 1, 'action': 1, 'all': 2, 'and': 4, 'apo...",No stakeholders identified in this statement.,Nitrates Action Programme,NaN,NaN,[]
1,671178d6b775f29fa839aeba,"joint committee on agriculture, food and the m...",443,2021,2020-2022,Chairman,The agenda for this morning’s meeting is the n...,"Agriculture, Food and Marine",gend morng meetg nitrtes ction progrmme consul...,"{'20': 1, 'about': 1, 'absolute': 2, 'action':...","{'20': 1, 'a': 6, 'about': 1, 'absolute': 2, '...",1. **Stakeholder Name**: Irish Farmers Associa...,new nitrates action programme,NaN,NaN,"[{'Stakeholder': 'Irish Farmers Association', ..."
2,671178d6b775f29fa839aebb,"joint committee on agriculture, food and the m...",443,2021,2020-2022,Mr. Tim Cullinan,I compliment the Chairman on organising this e...,"Agriculture, Food and Marine",compliment Chirmn event morng be cuse time sub...,"{'10': 1, '14': 1, '16': 1, '2005': 1, '2008':...","{'1': 1, '10': 1, '14': 1, '16': 1, '2': 4, '2...",1. **Stakeholder Name**: Farmers \n **Categ...,,NaN,NaN,"[{'Stakeholder': 'Farmers', 'Category': 'Produ..."
3,671178d6b775f29fa839aebc,"joint committee on agriculture, food and the m...",443,2021,2020-2022,Mr. Tim Cullinan,We are moving in the wrong direction there. W...,"Agriculture, Food and Marine",direction srge tnks sue compulsory low emsion ...,"{'100': 1, '15': 1, '30': 1, 'able': 1, 'above...","{'100': 1, '15': 1, '30': 1, 'a': 7, 'able': 1...",1. **Stakeholder Name**: Department\n **Cate...,"targeted agricultural modernisation scheme, TA...",NaN,NaN,"[{'Stakeholder': 'Department', 'Category': 'Re..."
4,671178d6b775f29fa839aebd,"joint committee on agriculture, food and the m...",443,2021,2020-2022,Mr. Pat McCormack,I thank committee members for the opportunity ...,"Agriculture, Food and Marine",committee members opportunity views behlf CMSA...,"{'000': 4, '10': 1, '100': 2, '12': 2, '120': ...","{'000': 4, '10': 1, '100': 2, '12': 1, '12-mon...",1. **Stakeholder Name**: ICMSA\n **Category*...,"TAMS, agricultural sustainability support and ...",NaN,NaN,"[{'Stakeholder': 'ICMSA', 'Category': 'Product..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19241,672bde88aa798a5c53f9b4c8,"dáil éireann debate - wednesday, 19 may 2021",415,2021,2020-2022,Deputy Kathleen Funchion,"I move amendment No. 14: In page 8, between li...",Dail Eireann,NaN,NaN,"{'13': 1, '14': 2, '15': 1, '20': 1, '21': 1, ...",No stakeholders identified in this statement.,,Gender Pay Gap Information Bill 2019: Report a...,"I move amendment No. 14: In page 8, between li...",[]
19242,672bde89aa798a5c53f9b4c9,"dáil éireann debate - wednesday, 19 may 2021",415,2021,2020-2022,Deputy Roderic O’Gorman,I am of the view that the strengthening we hav...,Dail Eireann,NaN,NaN,"{'13': 1, 'a': 6, 'accept': 2, 'actions': 1, '...",No stakeholders identified in this statement.,,Gender Pay Gap Information Bill 2019: Report a...,I am of the view that the strengthening we hav...,[]
19243,672bde89aa798a5c53f9b4ca,"dáil éireann debate - wednesday, 19 may 2021",415,2021,2020-2022,Deputy Kathleen Funchion,I will withdraw amendments Nos. 14 and 15. Ame...,Dail Eireann,NaN,NaN,"{'14': 1, '15': 2, 'amendment': 2, 'amendments...",No stakeholders identified in this statement.,,Gender Pay Gap Information Bill 2019: Report a...,I will withdraw amendments Nos. 14 and 15. Ame...,[]
19244,672bde89aa798a5c53f9b4cb,"dáil éireann debate - wednesday, 19 may 2021",415,2021,2020-2022,Deputy Roderic O’Gorman,"I move amendment No. 16: In page 

In [15]:
import ast

def flatten_stakeholder_data(data, speaker_column = "Speaker", relation_column = "Stakeholder_Category"):
    flattened_data = []
    #data[relation_column] = data[relation_column].apply(lambda x: x if isinstance(x, list) else [])
    for _,row in data.iterrows():
        speaker = row[speaker_column]
        exacttext = row['Exact Text']
        relations = row[relation_column]

        if isinstance(relations, str):
            try:
                relations = ast.literal_eval(relations)
            except (ValueError, SyntaxError) as e:
                relations = []
                
        if isinstance(relations, list) and relations:
            for relation in relations:
                flattened_data.append({
                    "Speaker":speaker,
                    "Exact Text": exacttext,
                    "Stakeholder":relation.get("Stakeholder",""),
                    "Category":relation.get("Category",""),
                    "Mention":relation.get("Mention","")
                })
        else:
            continue
    return pd.DataFrame(flattened_data)

In [16]:
flattened_stakeholder = flatten_stakeholder_data(data)
print(flattened_stakeholder)

                    Speaker  \
0                  Chairman   
1                  Chairman   
2                  Chairman   
3                  Chairman   
4                  Chairman   
...                     ...   
1792  Deputy Thomas Pringle   
1793  Deputy Thomas Pringle   
1794  Deputy Thomas Pringle   
1795  Deputy Thomas Pringle   
1796   Deputy Jackie Cahill   

                                             Exact Text  \
0     The agenda for this morning’s meeting is the n...   
1     The agenda for this morning’s meeting is the n...   
2     The agenda for this morning’s meeting is the n...   
3     The agenda for this morning’s meeting is the n...   
4     The agenda for this morning’s meeting is the n...   
...                                                 ...   
1792  I thank the Leas-Cheann Comhairle for the oppo...   
1793  I thank the Leas-Cheann Comhairle for the oppo...   
1794  I thank the Leas-Cheann Comhairle for the oppo...   
1795  I thank the Leas-Cheann Comhair

In [17]:
flattened_stakeholder.columns = flattened_stakeholder.columns.str.strip()
flattened_stakeholder.to_excel(r'stake.xlsx')

In [18]:
def summarise(data):
    thoughts = []
    
    for _, row in data.iterrows():
        speaker = row['Speaker']
        stakeholder = row['Stakeholder']
        stakeholder_category = row['Category']
        exact_text = row['Exact Text']
     
        system_prompt = (
            "You are a tool that summarises, in one line, the speaker's position on each of the listed stakeholder organisations, "
            "that have been extracted from the attached text.\n"
            "When processing a stakeholder, return the results as a one-line summary.\n"
            "Each summary must reflect the speaker's stance on the stakeholder, in a positive, neagtive, or neutral manner. "
            "Ensure each summary is concise, specific, and does not include additional context beyond what is explicitly stated in the text. "
            "If a stakeholder is not discussed in the text or if the stakeholder is addressed without any meaningful or strongly expressed stance, indicate 'No position expressed' for that stakeholder."
        )
        
        user_prompt = (
            f"The attached list comprises stakeholder organisations belonging to specific categories extracted from the source text. "
            f"For each item in the list, provide a one-line summary of the position expressed by the speaker of the text in relation to the item, agriculture, and sustainability. "
            f"Refer specifically to the text only and do not generalise beyond the statements in the text.\n"
            f"Do not include any feedback in your answer.\n\n"
            f"Speaker: {speaker}\n"
            f"Stakeholder: {stakeholder}\n"
            f"Stakeholder Category: {stakeholder_category}\n"
            f"Context: {exact_text}\n\n"
            "Analyze the speaker's stance on the stakeholder mentioned and provide:\n"
            "1. The extracted thought or summary of their stance in the format below. Do not mention the speaker's name in the thoughts.\n"
            "Speaker Thoughts:"
        )
             
        response = openai.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=400,  
            temperature=0.2
        )
    
        result = response.choices[0].message.content.strip()
        thoughts.append(result)

    data['Thoughts'] = thoughts
    return data

In [19]:
thoughts_data = summarise(flattened_stakeholder)

In [20]:
def extract_sentiment_from_thoughts(df):
    system_prompt = (
        "You are a sentiment analysis assistant. Your task is to classify the sentiment of a speaker's thought "
        "on a specific Stakeholder into one of three categories: Positive, Negative, or Neutral. The sentiment should be "
        "based only on the given thought, and the response must be a single word: Positive, Negative, or Neutral."
    )
    
    sentiments = []

    for _, row in df.iterrows():
        speaker = row['Speaker']
        policy = row["Stakeholder"]
        thought = row["Thoughts"]

        user_prompt = (
            f"Policy: {policy}\n"
            f"Thought: {thought}\n\n"
            "Based on the thought, identify the sentiment as one of the following options:\n"
            "1. Positive: If the speaker's words express agreement, endorsement, or positivity toward the Stakeholder.\n"
            "2. Negative: If the speaker's words express disagreement, challenges, rejection, or negativity toward the Stakeholder.\n"
            "3. Neutral: If the speaker's words do not clearly indicate support or opposition.\n\n"
            "Provide your response as exactly one word: Positive, Negative, or Neutral."
        )

        response = openai.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=400,  
            temperature=0.2
        )

        sentiment = response.choices[0].message.content.strip()
        sentiments.append(sentiment)
    
    df['Sentiment'] = sentiments
    return df


In [21]:
sentiment_data = extract_sentiment_from_thoughts(thoughts_data)

In [22]:
for index, row in sentiment_data.iterrows():
    sentiment_data.at[index, 'Thoughts'] = row['Thoughts'].replace('Speaker Thoughts: ', '')

filtered_data = sentiment_data[sentiment_data['Thoughts'].notna() & ~sentiment_data['Thoughts'].str.contains('No position expressed')]

In [23]:
filtered_data

,Speaker,Exact Text,Stakeholder,Category,Mention,Thoughts,Sentiment
0,Chairman,The agenda for this morning’s meeting is the n...,Irish Farmers Association,Production - Associations Category,"""I welcome the following witnesses to the comm...",The speaker welcomes the Irish Farmers Associa...,Positive
5,Mr. Tim Cullinan,I compliment the Chairman on organising this e...,Farmers,Production Category,"""Farmers have put in excess of €2.2 billion of...",Farmers are crucial to the economy and have ma...,Neutral
6,Mr. Tim Cullinan,I compliment the Chairman on organising this e...,Department,Research and Regulatory Category,"""we must ensure Department officials, when the...",The Department is criticized for submitting la...,Negative
7,Mr. Tim Cullinan,I compliment the Chairman on organising this e...,Environmental Protection Agency (EPA),Research and Regulatory Category,"""we have great concern around water quality an...",The Environmental Protection Agency (EPA) is v...,Neutral
8,Mr. Tim Cullinan,We are moving in the wrong direction there. W...,Department,Research and Regulatory Category,"""I understand the Department is looking at the...",The Department is seen as moving in the wrong ...,Negative
...,...,...,...,...,...,...,...
1792,Deputy Thomas Pringle,I thank the Leas-Cheann Comhairle for the oppo...,Irish Farmers Association (IFA),Production - Associations Category,"""The IFA is still worried about families that ...",The IFA's concerns regarding the Bill's provis...,Neutral
1793,Deputy Thomas Pringle,I thank the Leas-Cheann Comhairle for the oppo...,Health Service Executive (HSE),Research and Regulatory Category,"""The executive will determine whether the appl...",The Health Service Executive (HSE) is viewed a...,Negative
1794,Deputy Thomas Pringle,I thank the Leas-Cheann Comhairle for the oppo...,Nursing Homes Ireland,Research and Regulatory Category,"""accessed instead through Nursing Homes Ireland.""",Nursing Homes Ireland is seen as a necessary e...,Neutral
1795,Deputy Thomas Pringle,I thank the Leas-Cheann Comhairle for the oppo...,Comptroller and Auditor General,Research and Regulatory Category,"""The Comptroller and Auditor General’s report ...",The Comptroller and Auditor General's report h...,Neutral


In [24]:
def insert_dataframe_into_mongodb(mongo_uri, db_name, collection_name, df):
    client = MongoClient(mongo_uri)
    db = client[db_name]
    collection = db[collection_name]

    data = df.to_dict(orient="records")

    if data:
        result = collection.insert_many(data)
        print(f"{len(result.inserted_ids)} documents inserted successfully.")
        return len(result.inserted_ids)
    else:
        print("No data to insert.")
        return 0

In [25]:
excel_data = filtered_data[['Speaker', 'Stakeholder', 'Category', 'Mention', 'Thoughts', 'Sentiment']]
excel_data = excel_data.drop_duplicates()
insert_dataframe_into_mongodb('mongodb://localhost:27017/', 'transcripts', 'stakeholders', excel_data)

1698 documents inserted successfully.


1698

In [34]:
excel_data.to_excel(r'stakeholder.xlsx')